In [13]:
import tensorflow as tf
import numpy as np
from memory import Memory
from basics import Basics
from tqdm import tqdm


from networks_kennedy import QN_l1, QN_guess_kennedy
import give_probability_kennedy




basics = Basics(layers=1)
basics.define_actions()
b =np.load("buffer.npy", allow_pickle=True)
buffer = Memory(len(b), load_path=None)
for i in range(len(b)):
    buffer.add_sample(tuple(b[i]))


In [2]:
q= np.zeros((2,11,2))
n= np.zeros((2,11,2))
q0= np.zeros((11))
n0= np.zeros((11))

qalph= np.zeros((2,11,2))
q0alph= np.zeros((11))

alpha = 1/buffer.num_samples

for v in buffer.samples:
    n[tuple([int(v[0]), int(v[2]),int(v[4])])] += 1
    nn= n[tuple([int(v[0]), int(v[2]),int(v[4])])]
    q[tuple([int(v[0]), int(v[2]),int(v[4])])] =  (1- (1/nn))*q[tuple([int(v[0]), int(v[2]),int(v[4])])] + (v[5])/nn

    n0[tuple([int(v[2])])] += 1
    nn0= n0[tuple([int(v[2])])] 
    q0[tuple([int(v[2])])] =  (1- (1/nn0))*q0[tuple([int(v[2])])] + np.max(q[int(v[0]), int(v[2]), :])/nn0
    
    
    qalph[tuple([int(v[0]), int(v[2]),int(v[4])])] =  (1- alpha)*qalph[tuple([int(v[0]), int(v[2]),int(v[4])])] + (v[5])*alpha
    q0alph[tuple([int(v[2])])] =  (1- alpha)*q0alph[tuple([int(v[2])])] + np.max(qalph[int(v[0]), int(v[2]), :])*alpha


In [16]:
tf.keras.backend.set_floatx('float64')

In [19]:

qvalues1 = qn_l1_prim(np.expand_dims(np.array([]), axis=0))
qvalues1targ = qn_l1_targ(np.expand_dims(np.array([]), axis=0))
qvalueslayerguess_prim = qn_guess_prim(np.expand_dims(np.array([1,-.7]), axis=0))
qvalueslayerguess_targ = qn_guess_targ(np.expand_dims(np.array([1,-.7]), axis=0))


In [20]:
buffer.num_samples

1000000

In [21]:
qvalues1

<tf.Tensor: shape=(1, 11), dtype=float64, numpy=
array([[0.        , 0.0851733 , 0.        , 0.02103042, 0.46062079,
        0.        , 0.        , 0.        , 0.05550656, 0.4593089 ,
        0.        ]])>

In [33]:
TAU = 0.1
batch_length = int(buffer.num_samples/10**2)
optimizers = [tf.keras.optimizers.SGD(lr = 1/buffer.num_samples), tf.keras.optimizers.SGD(lr = 1/buffer.num_samples)]
time=1


qn_l1_prim = QN_l1(basics.actions[0])
qn_l1_targ = QN_l1(basics.actions[0])
qn_guess_prim = QN_guess_kennedy(basics.possible_phases)
qn_guess_targ = QN_guess_kennedy(basics.possible_phases)



for _ in tqdm(range(100)):

    batch = buffer.sample(batch_length)


    for v in batch:
        outcome_1_beta1_batch = np.array([[ int(v[0]), v[1]] for v in batch ] )
        labels_beta1 = np.array([int(v[2]) for v in batch])


        q_guess_prim = qn_guess_prim(np.expand_dims(outcome_1_beta1_batch, axis=0))
        q_guess_prim = np.squeeze(q_guess_prim.numpy())

        opt_a_2_prim = np.argmax(q_guess_prim,axis=1)

        update_for_q_1_prim = qn_l1_targ(np.expand_dims(np.array([[] for i in range(len(batch))]), axis=0)) #targ = target
        update_for_q_1_prim = np.squeeze(update_for_q_1_prim, axis=0)
        qlabels_l1 = update_for_q_1_prim.copy()

        np.save("labelsbeta1", labels_beta1)
        qlabels_l1[np.arange(batch_length), labels_beta1] = np.squeeze(qn_guess_targ(np.expand_dims(outcome_1_beta1_batch, axis=0)).numpy())[np.arange(batch_length),opt_a_2_prim]

        optimizer_ql1, optimizer_guess = optimizers

        train_vars_in = [ch.numpy() for ch in qn_l1_prim.trainable_variables]

        
        s_3_batch = np.array([[v[0], v[1] ] for v in batch])
        rewards = np.array([v[-1] for v in batch])
        labels_guess = np.array([int(v[-2]) for v in batch])

        update_for_q_3_prim = qn_guess_targ(np.expand_dims(s_3_batch, axis=0))
        update_for_q_3_prim = np.squeeze(update_for_q_3_prim, axis=0)
        qlabels_l3 = update_for_q_3_prim.copy()
        qlabels_l3[np.arange(batch_length), labels_guess] = rewards[np.arange(batch_length)]
        
        # print(qlabels_l1)
        with tf.device("/cpu:0"):
            with tf.GradientTape() as tape:
                tape.watch(qn_l1_prim.trainable_variables)
                pred_q_1s = qn_l1_prim(np.expand_dims(np.array([[] for i in range(len(batch))]), axis=0))

                loss_sum =tf.keras.losses.MSE(pred_q_1s, qlabels_l1)
                loss = tf.reduce_mean(loss_sum)
                grads = tape.gradient(loss, qn_l1_prim.trainable_variables)
                optimizer_ql1.apply_gradients(zip(grads, qn_l1_prim.trainable_variables))

            with tf.GradientTape() as tape:
                tape.watch(qn_guess_prim.trainable_variables)
                pred_q_3s = qn_guess_prim(np.expand_dims(s_3_batch, axis=0))
                loss_sum =tf.keras.losses.MSE(pred_q_3s, qlabels_l3)
                loss = tf.reduce_mean(loss_sum)
                loss_guess = loss
                grads = tape.gradient(loss, qn_guess_prim.trainable_variables)
                optimizer_guess.apply_gradients(zip(grads, qn_guess_prim.trainable_variables))

        for t, e in zip(qn_l1_targ.trainable_variables, qn_l1_prim.trainable_variables):
            t.assign(t*(1-TAU) + e*TAU)

        for t, e in zip(qn_guess_targ.trainable_variables, qn_guess_prim.trainable_variables):
            t.assign(t*(1-TAU) + e*TAU)
        time+=10
        #for opt in optimizers:
         #   opt.learning_rate.value = np.max(0.01, np.exp(-time/100))



  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [40]:
for i in optimizer_guess.get_weights():
    print(i)

960


In [47]:
optimizer_guess.set_weights("learning_rate" 0.01)

TypeError: set_weights() takes 2 positional arguments but 3 were given

In [46]:
optimizer_guess.get_config()["learning_rate"]

1e-06

In [45]:
optimizer_guess.get_config()["learning_rate"]

1e-06

In [25]:

qvalues1 = qn_l1_prim(np.expand_dims(np.array([]), axis=0))
qvalues1targ = qn_l1_targ(np.expand_dims(np.array([]), axis=0))
qvalueslayerguess_prim = qn_guess_prim(np.expand_dims(np.array([1,-.7]), axis=0))
qvalueslayerguess_targ = qn_guess_targ(np.expand_dims(np.array([1,-.7]), axis=0))


In [26]:
qvalues1

<tf.Tensor: shape=(1, 11), dtype=float64, numpy=
array([[0.        , 0.09289872, 0.        , 0.02944692, 0.46494426,
        0.        , 0.        , 0.        , 0.06340189, 0.46419198,
        0.        ]])>